In [2]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.7.1
tiktoken version: 0.9.0


In [3]:
import os 
import urllib.request 


if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

len(raw_text)

20479

In [5]:
import re 

text = "Hello, world. This, is a test."

result=re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
result=re.split(r'([,.]|\s)', text)

result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [7]:
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [8]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

len(raw_text)

20479

In [9]:
text="Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result=[item.strip() for item in result if item.strip()]
len(result)

4690

In [10]:
all_words=sorted(set(result))
vocab_size=len(all_words)
print(vocab_size)

1130


In [11]:
vocab={token:integer for integer, token in enumerate(all_words)}

In [12]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        preprocessed=[
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [13]:
tokenizer = SimpleTokenizerV1(vocab)

In [14]:
text="It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [15]:
ids=tokenizer.encode(text)

In [16]:
tokenizer.decode(ids)

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride."

In [17]:
text="Hello, do you like tea. is this-- a test?"

all_tokens=sorted(list(set(result)))

all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer, token in enumerate(all_tokens)}

len(vocab.items())

1132

In [18]:
tokenizer2=SimpleTokenizerV1(vocab)

tokenizer2.encode(text)
tokenizer2.decode(tokenizer2.encode(text))

'<|unk|>, do you like tea. is this -- a <|unk|>?'

In [19]:
import tiktoken

tiktoken.__version__


'0.9.0'

In [20]:
tokenizer=tiktoken.get_encoding("gpt2")

tokenizer.encode("Hello world")

[15496, 995]

In [21]:
tokenizer.decode(tokenizer.encode("Hello world"))

'Hello world'

In [22]:
text = ( 
    "Hello, do you like tea? <|endoftext|> fsdf sdf sdfIn the sunlit terraces"
    "of someunknownPlace."
)

tokenizer.encode(text, allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 43458,
 7568,
 264,
 7568,
 264,
 7568,
 818,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

In [23]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [24]:
enc_sample=enc_text[50:]

In [25]:
enc_sample=enc_text[50:]

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")



x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [26]:
import torch

In [27]:
torch.__version__

'2.7.1+cu126'

In [28]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



In [29]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader



In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text=f.read()

In [31]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

first_batch = next(data_iter)
print(first_batch)



[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [32]:
input_ids=torch.tensor([1,2,3,4])


vocab_size=6
output_dim=3

torch.manual_seed(123)
embedding_layer=torch.nn.Embedding(tokenizer.n_vocab, output_dim)

In [33]:
embedding_layer(torch.tensor([3]))

tensor([[-1.1925,  0.6984, -1.4097]], grad_fn=<EmbeddingBackward0>)

In [34]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [35]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)



In [37]:
token_embeddings = token_embedding_layer(inputs)

token_embeddings.shape

torch.Size([8, 4, 256])

In [38]:
token_embeddings[0, 0]

tensor([ 4.5925e-01,  7.6589e-01, -1.7269e+00,  2.7519e-01,  2.3155e-01,
        -4.7319e-01, -3.9847e-01,  1.7265e+00,  8.9715e-01, -1.0197e-01,
         5.1204e-01, -1.6805e+00, -8.0522e-01,  1.1375e+00, -9.9466e-01,
        -9.5338e-01,  8.3238e-01,  1.3210e+00,  9.5062e-01,  1.2799e+00,
         1.1932e+00,  6.7199e-01, -6.2126e-01,  5.5973e-01, -6.0924e-01,
        -9.0394e-02, -1.4039e+00, -4.1480e-01,  2.5625e-01, -9.7720e-03,
        -2.2002e+00, -9.0485e-01, -4.2069e-01, -4.4801e-01,  9.3138e-01,
         2.4970e+00, -7.5886e-01, -2.9304e-03,  4.6033e-01, -1.2624e+00,
         3.7764e-01,  8.1343e-01, -1.2823e+00, -1.2091e-01, -3.3686e-01,
         1.6070e+00, -5.9332e-01, -1.6217e+00,  1.2313e+00, -1.8392e+00,
        -1.2068e+00, -2.5405e+00,  1.9803e+00, -1.2905e+00,  1.7485e+00,
         8.1073e-01,  1.1342e+00, -1.7977e+00, -1.0314e+00, -1.2450e+00,
        -8.8608e-01,  7.4233e-02,  1.0778e+00, -4.8303e-01, -5.6789e-01,
         7.1249e-01,  8.5286e-01,  7.1491e-01, -1.0

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)



In [41]:
torch.arange(max_length)

pos_embeddings=pos_embedding_layer(torch.arange(max_length))

input_embeddings=token_embeddings+pos_embeddings